# Calculate average precipitation rate of MCSs

In [1]:
import pathlib

import numpy as np
import xarray as xr

MCS by day

Steps:
1. Make blobs binary (1 or 0)
2. Take sum of binary blobs for each day
    1. For any days with sum > 3 (over 3 hours of MCS rain), set it as 1
    2. Otherwise, 0
3. Take sum of precipitation for each day
4. Multiply by MCS days from step 2
5. Sum and divide by 12 and multiply by (3600 * 1000), where 12 is number of years
6. To get MCS precip %, divide these results by the JJ avg prect (can be the monthly file multiplied by 61)

### Calculate binary blobs

### Group binary blobs by day:

In [2]:
data_dir = pathlib.Path("/glade/derecho/scratch/malbright/nam_mcs_tracking/historical")
files = sorted(data_dir.glob("historical_blobs_*.compressed.nc"))
files

[PosixPath('/glade/derecho/scratch/malbright/nam_mcs_tracking/historical/historical_blobs_1991.compressed.nc'),
 PosixPath('/glade/derecho/scratch/malbright/nam_mcs_tracking/historical/historical_blobs_1992.compressed.nc'),
 PosixPath('/glade/derecho/scratch/malbright/nam_mcs_tracking/historical/historical_blobs_1993.compressed.nc'),
 PosixPath('/glade/derecho/scratch/malbright/nam_mcs_tracking/historical/historical_blobs_1994.compressed.nc'),
 PosixPath('/glade/derecho/scratch/malbright/nam_mcs_tracking/historical/historical_blobs_1995.compressed.nc'),
 PosixPath('/glade/derecho/scratch/malbright/nam_mcs_tracking/historical/historical_blobs_1996.compressed.nc'),
 PosixPath('/glade/derecho/scratch/malbright/nam_mcs_tracking/historical/historical_blobs_1997.compressed.nc'),
 PosixPath('/glade/derecho/scratch/malbright/nam_mcs_tracking/historical/historical_blobs_1998.compressed.nc'),
 PosixPath('/glade/derecho/scratch/malbright/nam_mcs_tracking/historical/historical_blobs_1999.compresse

In [3]:
path = "/glade/derecho/scratch/malbright/nam_mcs_tracking/historical/"
for file in files:
    # Read in file
    print(f"Processing {file}")
    ds = (
        xr.open_dataset(file, engine="netcdf4")
        .sel(lat=slice(5, 55))
        .sel(lon=slice(235, 295))
    )
    # Convert hourly data to daily
    ds = ds.resample(time="D").sum()  # where D is for days
    # set days with MCS as 1, otherwise 0
    ds = xr.where(ds > 0, 1, 0)

    # save results
    name = file.stem.split("/")

    fname = (
        str(name[0]) + "_binary_days"
    )  # str(name[2]) + '_' + str(name[3]) + '_' + str(name[4])

    print("Exporting...")
    ds.to_netcdf(path + fname + ".nc")
    print("Saved!")

Processing /glade/derecho/scratch/malbright/nam_mcs_tracking/historical/historical_blobs_1991.compressed.nc
Exporting...
Saved!
Processing /glade/derecho/scratch/malbright/nam_mcs_tracking/historical/historical_blobs_1992.compressed.nc
Exporting...
Saved!
Processing /glade/derecho/scratch/malbright/nam_mcs_tracking/historical/historical_blobs_1993.compressed.nc
Exporting...
Saved!
Processing /glade/derecho/scratch/malbright/nam_mcs_tracking/historical/historical_blobs_1994.compressed.nc
Exporting...
Saved!
Processing /glade/derecho/scratch/malbright/nam_mcs_tracking/historical/historical_blobs_1995.compressed.nc
Exporting...
Saved!
Processing /glade/derecho/scratch/malbright/nam_mcs_tracking/historical/historical_blobs_1996.compressed.nc
Exporting...
Saved!
Processing /glade/derecho/scratch/malbright/nam_mcs_tracking/historical/historical_blobs_1997.compressed.nc
Exporting...
Saved!
Processing /glade/derecho/scratch/malbright/nam_mcs_tracking/historical/historical_blobs_1998.compressed

### Group PRECT by day, then multiply with binary blob days

In [2]:
data_dir_blobs = pathlib.Path("/glade/derecho/scratch/malbright/nam_mcs_tracking/historical/")
b_files = sorted(data_dir_blobs.glob("historical_blobs_*.compressed_binary_days.nc"))
len(b_files)

13

In [3]:
p_path = pathlib.Path("/glade/derecho/scratch/malbright/prect_nam_mcs_hourly1/historical")
p_files = sorted(p_path.glob("b.e13.B20TRC5CN.ne120_g16.002.cam.h5.PRECT.*.remap.nc"))
len(p_files)

13

In [4]:
path = "/glade/derecho/scratch/malbright/nam_mcs_tracking/historical/"
for i in range(len(b_files)):
    # Read in files
    ds_b = xr.open_dataset(b_files[i], engine="netcdf4")
    ds_p = (
        xr.open_dataset(p_files[i])
        .sel(lat=slice(5, 55))
        .sel(lon=slice(235, 295))
    )
    year = np.unique(ds_b.time.dt.year)[0]
    print(f"Processing year {year}")
    # ds_p = ds_p.sel(time=is_month(ds_p['time.month']))
    # resample prect
    ds_p = ds_p.resample(time="D").sum()  # where D is for days
    # Compute mcs rain rate
    ds = ds_b["MCS_PRECT"] * ds_p["PRECT"]

    # save results
    ds.to_netcdf(
        f"{path}mcs_day_prect_historical_{year}.nc"
    )

Processing year 1991
Processing year 1992
Processing year 1993
Processing year 1994
Processing year 1995
Processing year 1996
Processing year 1997
Processing year 1998
Processing year 1999
Processing year 2000
Processing year 2001
Processing year 2002
Processing year 2003


### Calculate MCS precip % by day

In [6]:
data_dir_blobs = pathlib.Path("/glade/derecho/scratch/malbright/nam_mcs_tracking/historical/")
b_files = sorted(data_dir_blobs.glob("mcs_day_prect_historical_*.nc"))
len(b_files)

13

In [7]:
p_path = pathlib.Path("/glade/derecho/scratch/malbright/prect_nam_mcs_hourly1/historical")
p_files = sorted(p_path.glob("b.e13.B20TRC5CN.ne120_g16.002.cam.h5.PRECT.*.remap.nc"))
len(p_files)

13

In [ ]:
path = "/glade/derecho/scratch/malbright/nam_mcs_tracking/historical/"
for i in range(len(b_files)):
    
    # Read in files
    ds_b = xr.open_dataset(b_files[i], engine="netcdf4")

    ds_b_total = ds_b.mean(dim='time') * len(ds_b.time)

    ds_p = (
        xr.open_dataset(p_files[i])
        .sel(lat=slice(5, 55))
        .sel(lon=slice(235, 295))
    )

    year = np.unique(ds_b.time.dt.year)[0]
    print(f"Processing year {year}")
    # ds_p = ds_p.sel(time=is_month(ds_p['time.month']))
    # resample prect
    ds_p = ds_p.resample(time="D").sum()  # where D is for days
    ds_p_total = ds_p.mean(dim='time') * len(ds_p.time)

    # Compute mcs rain rate %
    ds = ds_b_total["__xarray_dataarray_variable__"] / ds_p_total["PRECT"] * 100
    ds = ds.rename("mcs_precip_percentage")

    # save results
    ds.to_netcdf(
        f"{path}mcs_precip_percent_historical_{year}.nc"
    )

Processing year 1991
Processing year 1992
Processing year 1993
Processing year 1994
Processing year 1995
Processing year 1996
Processing year 1997
Processing year 1998
Processing year 1999
Processing year 2000


In [9]:
path = "/glade/derecho/scratch/malbright/nam_mcs_tracking/historical/"
for i in range(len(b_files)):
    
    # Read in files
    ds_b = xr.open_dataset(b_files[i], engine="netcdf4")

    ds_b = ds_b.sel(time=ds_b.time.dt.month.isin([6, 7, 8]))
    ds_b_total = ds_b.mean(dim='time') * len(ds_b.time)

    ds_p = (
        xr.open_dataset(p_files[i])
        .sel(lat=slice(5, 55))
        .sel(lon=slice(235, 295))
    )

    year = np.unique(ds_b.time.dt.year)[0]
    print(f"Processing year {year}")
    # ds_p = ds_p.sel(time=is_month(ds_p['time.month']))
    # resample prect
    ds_p = ds_p.sel(time=ds_p.time.dt.month.isin([6, 7, 8]))
    ds_p = ds_p.resample(time="D").sum()  # where D is for days
    ds_p_total = ds_p.mean(dim='time') * len(ds_p.time)

    # Compute mcs rain rate %
    ds = ds_b_total["__xarray_dataarray_variable__"] / ds_p_total["PRECT"] * 100
    ds = ds.rename("mcs_precip_percentage")

    # save results
    ds.to_netcdf(
        f"{path}mcs_precip_percent_historical_{year}_JJA.nc"
    )

Processing year 1991
Processing year 1992
Processing year 1993
Processing year 1994
Processing year 1995
Processing year 1996
Processing year 1997
Processing year 1998
Processing year 1999
Processing year 2000
Processing year 2001
Processing year 2002
Processing year 2003


## Old:

In [7]:
mcs_precip_days = xr.open_mfdataset(
    "/glade/scratch/malbright/mcs_tracking/pliocene/mcs_rate/blobs*_mintime4_minsize6_overlap10-90_jj_day_prect.nc"
).load()

In [8]:
mcs_precip_days_avg = mcs_precip_days.sum(dim="time") / 13 * (3600 * 1000)

In [9]:
mcs_precip_days_avg.to_netcdf(
    "/glade/scratch/malbright/mcs_tracking/pliocene/mcs_rate/mcs_day_avg_plio.nc"
)

In [10]:
prect1 = xr.open_mfdataset(
    "/glade/scratch/malbright/hourly1/plio_jj/b.e13.B1850C5CN.ne120_g16.pliohiRes.002.cam.h5.PRECT.*_jj.remap.nc"
).load()

In [12]:
prect_days = prect1.resample(time="D").sum()

In [13]:
prect_rate = prect_days.PRECT.mean(dim="time") * 3600 * 61 * 1000

In [14]:
percent = mcs_precip_days_avg / prect_rate * 100

In [15]:
percent.to_netcdf(
    "/glade/scratch/malbright/mcs_tracking/pliocene/mcs_rate/mcs_day_percent_plio.nc"
)

### Group PRECT by day:

In [ ]:
prect = xr.open_mfdataset(
    "/glade/scratch/malbright/remap/b.e13.B1850C5CN.ne120_g16.pliohiRes.002.cam.h5.PRECT.*.nc",
    parallel=True,
)
prect = prect.compute()

In [ ]:
prect_days = prect.resample(time="D").sum()

### Multiply blob days by PRECT days, take sum, and divide by number of years:

In [ ]:
mcs_precip_days = b_days_mcs_precip.MCS_PRECT * prect_days.PRECT

In [ ]:
mcs_precip_days_avg = mcs_precip_days.sum(dim="time") / 13 * (3600 * 1000)

### Calculate percentage:

In [ ]:
prect_rate = prect_days.PRECT.mean(dim="time") * 3600 * 61 * 1000

In [ ]:
percent = mcs_precip_days_avg / h_prec * 100

### Compress data:

In [2]:
data_dir = pathlib.Path(
    "/glade/derecho/scratch/malbright/nam_mcs_tracking/historical"
)
files = sorted(
    data_dir.glob(
        "historical_blobs_*.nc"
    )
)
files

[PosixPath('/glade/derecho/scratch/malbright/nam_mcs_tracking/historical/historical_blobs_1991.nc'),
 PosixPath('/glade/derecho/scratch/malbright/nam_mcs_tracking/historical/historical_blobs_1992.nc'),
 PosixPath('/glade/derecho/scratch/malbright/nam_mcs_tracking/historical/historical_blobs_1993.nc'),
 PosixPath('/glade/derecho/scratch/malbright/nam_mcs_tracking/historical/historical_blobs_1994.nc'),
 PosixPath('/glade/derecho/scratch/malbright/nam_mcs_tracking/historical/historical_blobs_1995.nc'),
 PosixPath('/glade/derecho/scratch/malbright/nam_mcs_tracking/historical/historical_blobs_1996.nc'),
 PosixPath('/glade/derecho/scratch/malbright/nam_mcs_tracking/historical/historical_blobs_1997.nc'),
 PosixPath('/glade/derecho/scratch/malbright/nam_mcs_tracking/historical/historical_blobs_1998.nc'),
 PosixPath('/glade/derecho/scratch/malbright/nam_mcs_tracking/historical/historical_blobs_1999.nc'),
 PosixPath('/glade/derecho/scratch/malbright/nam_mcs_tracking/historical/historical_blobs_2

In [3]:
for file in files:
    ds = xr.open_dataset(file)

    comp = dict(zlib=True)
    encoding = {var: comp for var in ds.data_vars}

    print(f'Writing output for {file.stem}...')
    ds.to_netcdf(
        path=f"{file.parent}/{file.stem}.compressed.nc",
        mode='w',
        format='NETCDF4',
        unlimited_dims='time',
        encoding=encoding
    )
    print(f'Compressed file written to {file.parent}')

Writing output for historical_blobs_1991...
Compressed file written to /glade/derecho/scratch/malbright/nam_mcs_tracking/historical
Writing output for historical_blobs_1992...
Compressed file written to /glade/derecho/scratch/malbright/nam_mcs_tracking/historical
Writing output for historical_blobs_1993...
Compressed file written to /glade/derecho/scratch/malbright/nam_mcs_tracking/historical
Writing output for historical_blobs_1994...
Compressed file written to /glade/derecho/scratch/malbright/nam_mcs_tracking/historical
Writing output for historical_blobs_1995...
Compressed file written to /glade/derecho/scratch/malbright/nam_mcs_tracking/historical
Writing output for historical_blobs_1996...
Compressed file written to /glade/derecho/scratch/malbright/nam_mcs_tracking/historical
Writing output for historical_blobs_1997...
Compressed file written to /glade/derecho/scratch/malbright/nam_mcs_tracking/historical
Writing output for historical_blobs_1998...
Compressed file written to /glad